## Part C: Training and Deploying with the AWS SageMaker

In [ ]:
import tarfile
input_shape = [1,3,224,224]
trace = torch.jit.trace(resnet.float().eval(), torch.zeros(input_shape).float())
trace.save('model.pth')

with tarfile.open('model.tar.gz', 'w:gz') as f:
    f.add('model.pth')

In [ ]:
import boto3
import sagemaker
import time
from sagemaker.utils import name_from_base

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()

compilation_job_name = name_from_base('TorchVision-ResNet-Neo-2')
prefix = compilation_job_name+'/model'

model_path = sess.upload_data(path='model.tar.gz', key_prefix=prefix)

data_shape = '{"input0":[1,3,224,224]}'
target_device = 'ml_c5'
framework = 'PYTORCH'
framework_version = '1.4.0'
compiled_model_path = 's3://sagemaker-models-output'

In [ ]:
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.predictor import Predictor

sagemaker_model = PyTorchModel(model_data=model_path,
                               predictor_cls=Predictor,
                               framework_version = framework_version,
                               role=role,
                               sagemaker_session=sess,
                               entry_point='resnet50.py',
                               source_dir='code',
                               py_version='py3',
                               env={'MMS_DEFAULT_RESPONSE_TIMEOUT': '500'}
                              )

In [ ]:
compiled_model = sagemaker_model.compile(target_instance_family=target_device, 
                                         input_shape=data_shape,
                                         job_name=compilation_job_name,
                                         role=role,
                                         framework=framework.lower(),
                                         framework_version=framework_version,
                                         output_path=compiled_model_path
                                        )

In [ ]:
predictor = compiled_model.deploy(initial_instance_count = 1,
                                  instance_type = 'ml.c5.9xlarge'
                                 )

In [ ]:
import numpy as np
import json

with open('01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg', 'rb') as f:
    payload = f.read()
    payload = bytearray(payload) 

response = predictor.predict(payload)
result = json.loads(response.decode())
print('Most likely class: {}'.format(np.argmax(result)))

In [ ]:
sess.delete_endpoint(predictor.endpoint_name)